# 🎵 SMT (Sheet Music Transformer) - Treinamento Otimizado para Kaggle

Este notebook está otimizado para treinar o modelo SMT no Kaggle com:
- **2x GPUs Tesla P100/T4**
- **30GB RAM**
- **Otimizações de memória e performance**

## 📋 Configurações Aplicadas

✅ **Otimizações de Performance:**
- `num_proc=1` (evita overhead de multiprocessing)
- `num_workers=4` (balanceado para 2 GPUs)
- `batch_size=2` com gradient accumulation (batch efetivo = 16)
- Cache habilitado nos datasets
- Mixed precision training (16-bit)
- DDP Strategy para multi-GPU

✅ **Otimizações de Memória:**
- `reduce_ratio=0.5` (reduz tamanho das imagens)
- `writer_batch_size=100` (reduzido de 500)
- Garbage collection automático
- Gradient clipping habilitado

## 🚀 Vamos começar!

## 1️⃣ Instalação de Dependências

Primeiro, vamos instalar todas as bibliotecas necessárias.

In [ ]:
%%time
# Instalar dependências principais
!pip install -q fire gin_config lightning loguru musicdiff numpy opencv-python-headless Pillow
!pip install -q rich scikit-image timm torch torchinfo torchvision transformers datasets

# Instalar dependências para geração sintética
!pip install -q verovio cairosvg names wonderwords

# Verificar GPUs disponíveis
import torch
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
print(f"🎮 Number of GPUs: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

## 2️⃣ Clonar o Repositório e Preparar Ambiente

Vamos clonar o código do SMT e configurar o ambiente.

In [ ]:
# Clone o repositório e faça pull das últimas alterações
!git clone https://github.com/gasbiru/SMT.git
%cd SMT

# Atualizar com últimas alterações do remoto
!git pull origin main

# Criar diretórios necessários
!mkdir -p /kaggle/working/checkpoints
!mkdir -p vocab

# Verificar estrutura
!ls -la

## 3️⃣ Configuração Otimizada para Kaggle

Vamos criar a configuração JSON otimizada para o ambiente Kaggle.

In [ ]:
import json

# Configuração otimizada para Kaggle com 2 GPUs e 30GB RAM
kaggle_config = {
    "data": {
        "data_path": "antoniorv6/full-page-grandstaff",
        "batch_size": 2,  # Batch por GPU (total = 2*2 = 4)
        "vocab_name": "FP_GrandStaff_BeKern",
        "num_workers": 4,  # Otimizado para 2 GPUs
        "krn_format": "bekern",
        "reduce_ratio": 0.5  # Reduz imagens pela metade para economizar memória
    },
    "checkpoint": {
        "filename": "SMT-Kaggle-best",
        "monitor": "val_SER",
        "dirpath": "/kaggle/working/checkpoints/",
        "mode": "min",
        "save_top_k": 2,
        "verbose": True
    }
}

# Salvar configuração
with open('config/FP-GrandStaff/kaggle_config.json', 'w') as f:
    json.dump(kaggle_config, f, indent=2)

print("✅ Configuração criada:")
print(json.dumps(kaggle_config, indent=2))

## 4️⃣ Monitoramento de Recursos

Vamos verificar o uso de memória antes de começar o treinamento.

In [ ]:
import psutil
import gc

# Verificar RAM disponível
ram = psutil.virtual_memory()
print(f"💾 RAM Total: {ram.total / 1024**3:.2f} GB")
print(f"💾 RAM Disponível: {ram.available / 1024**3:.2f} GB")
print(f"💾 RAM Usada: {ram.percent}%")

# Limpar memória GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    
    for i in range(torch.cuda.device_count()):
        print(f"\n🎮 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Memória Alocada: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
        print(f"   Memória Reservada: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
        print(f"   Memória Livre: {(torch.cuda.get_device_properties(i).total_memory - torch.cuda.memory_allocated(i)) / 1024**3:.2f} GB")

## 5️⃣ Teste Rápido de Carregamento de Dados

Antes de treinar, vamos testar se o dataset carrega corretamente com as otimizações.

In [ ]:
%%time
import datasets
from data import prepare_fp_data

# Teste de carregamento com otimizações
print("📥 Testando carregamento do dataset...")
test_ds = datasets.load_dataset(
    "antoniorv6/full-page-grandstaff", 
    split="train[:10]",  # Apenas 10 amostras para teste
    trust_remote_code=False
)

print(f"✅ Dataset carregado: {len(test_ds)} amostras")
print(f"📊 Colunas: {test_ds.column_names}")

# Testar processamento com num_proc=1
print("\n🔄 Testando processamento com num_proc=1...")
processed_ds = test_ds.map(
    prepare_fp_data,
    fn_kwargs={"reduce_ratio": 0.5, "krn_format": "bekern"},
    num_proc=1,  # Otimizado!
    writer_batch_size=100,
    load_from_cache_file=True
)

print(f"✅ Processamento concluído!")
print(f"📏 Exemplo - Tamanho imagem: {processed_ds[0]['image'].shape}")
print(f"📝 Exemplo - Tamanho transcrição: {len(processed_ds[0]['transcription'])} tokens")

# Limpar memória
del test_ds, processed_ds
gc.collect()

## 6️⃣ Iniciar Treinamento (Opção 1: Script Otimizado)

Use o script otimizado `train_kaggle.py` que já inclui todas as configurações para 2 GPUs.

In [ ]:
# Treinar usando o script otimizado
# use_wandb=False para não usar Weights & Biases (economiza tempo)
# use_wandb=True se você tiver uma conta WandB configurada

!python train_kaggle.py \
    --config_path="config/FP-GrandStaff/kaggle_config.json" \
    --use_wandb=False \
    --max_epochs=50

## 7️⃣ Iniciar Treinamento (Opção 2: Código Inline)

Alternativamente, você pode treinar diretamente no notebook com controle total.

In [ ]:
import json
import torch
import gc
from data import SyntheticCLGrandStaffDataset
from smt_trainer import SMT_Trainer
from ExperimentConfig import experiment_config_from_dict
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.strategies import DDPStrategy

# Configurações
torch.set_float32_matmul_precision('high')
torch.cuda.empty_cache()
gc.collect()

# Carregar configuração
with open('config/FP-GrandStaff/kaggle_config.json', "r") as f:
    config = experiment_config_from_dict(json.load(f))

print("🚀 Iniciando treinamento otimizado para Kaggle...")
print(f"📊 Batch size: {config.data.batch_size} (efetivo: {config.data.batch_size * 2 * 4} com 2 GPUs e grad accum)")
print(f"👷 Workers: {config.data.num_workers}")
print(f"📐 Reduce ratio: {config.data.reduce_ratio}")

# Carregar dados
print("\n📥 Carregando dataset...")
datamodule = SyntheticCLGrandStaffDataset(config=config.data, skip_steps=0)

# Criar modelo
max_height = datamodule.get_max_height()
max_width = datamodule.get_max_width()
max_len = datamodule.get_max_length()

print(f"\n📏 Dimensões do modelo:")
print(f"   Height: {max_height}, Width: {max_width}, Length: {max_len}")

model_wrapper = SMT_Trainer(
    maxh=int(max_height), maxw=int(max_width), maxlen=int(max_len),
    out_categories=len(datamodule.train_set.w2i), 
    padding_token=datamodule.train_set.w2i["<pad>"],
    in_channels=1, w2i=datamodule.train_set.w2i, i2w=datamodule.train_set.i2w,
    d_model=256, dim_ff=256, num_dec_layers=8
)

# Callbacks
early_stopping = EarlyStopping(
    monitor="val_SER", min_delta=0.01, patience=10, mode="min", verbose=True
)

checkpointer = ModelCheckpoint(
    dirpath=config.checkpoint.dirpath,
    filename=config.checkpoint.filename,
    monitor=config.checkpoint.monitor,
    mode=config.checkpoint.mode,
    save_top_k=config.checkpoint.save_top_k,
    verbose=True,
    save_last=True
)

# Estratégia multi-GPU
strategy = DDPStrategy(
    find_unused_parameters=False,
    gradient_as_bucket_view=True
)

# Trainer otimizado
trainer = Trainer(
    max_epochs=50,
    check_val_every_n_epoch=2,
    callbacks=[checkpointer, early_stopping],
    precision='16-mixed',
    accelerator='gpu',
    devices=2,  # 2 GPUs
    strategy=strategy,
    accumulate_grad_batches=4,  # Batch efetivo = 2*2*4 = 16
    gradient_clip_val=1.0,
    log_every_n_steps=50,
    enable_progress_bar=True
)

print(f"\n🎮 Configuração do Trainer:")
print(f"   GPUs: {trainer.num_devices}")
print(f"   Batch efetivo: {config.data.batch_size * trainer.num_devices * trainer.accumulate_grad_batches}")
print(f"   Precisão: {trainer.precision}")

# Treinar!
print("\n🚀 Iniciando treinamento...\n")
trainer.fit(model_wrapper, datamodule=datamodule)

## 8️⃣ Avaliar Modelo

Após o treinamento, teste o melhor modelo.

In [ ]:
# Testar o melhor modelo
if checkpointer.best_model_path:
    print(f"🏆 Carregando melhor modelo: {checkpointer.best_model_path}")
    best_model = SMT_Trainer.load_from_checkpoint(checkpointer.best_model_path)
    
    print("\n🧪 Testando modelo...")
    test_results = trainer.test(best_model, datamodule=datamodule)
    
    print("\n✅ Resultados do teste:")
    for key, value in test_results[0].items():
        print(f"   {key}: {value:.4f}")
else:
    print("⚠️ Nenhum checkpoint encontrado")

## 9️⃣ Salvar e Baixar Checkpoints

Salve os checkpoints para uso posterior.

In [ ]:
# Listar checkpoints salvos
!ls -lh /kaggle/working/checkpoints/

# Comprimir checkpoints para download
!tar -czf checkpoints.tar.gz /kaggle/working/checkpoints/

print("✅ Checkpoints comprimidos em checkpoints.tar.gz")
print("📥 Você pode baixar o arquivo pela aba de Output do Kaggle")

## 🔟 Dicas e Troubleshooting

### 💡 Dicas para Otimização

1. **Se ficar sem memória:**
   - Reduza `batch_size` de 2 para 1
   - Aumente `reduce_ratio` para 0.3 (imagens menores)
   - Reduza `accumulate_grad_batches` de 4 para 2

2. **Se o Map ainda travar:**
   - Já está com `num_proc=1` (otimizado)
   - Cache habilitado com `load_from_cache_file=True`
   - Se travar novamente, tente limpar o cache: `!rm -rf ~/.cache/huggingface/datasets/`

3. **Para acelerar o treinamento:**
   - Use `check_val_every_n_epoch=5` (validar menos frequentemente)
   - Aumente `log_every_n_steps=100`
   - Desabilite checkpoints intermediários: `save_top_k=1`

4. **Monitoramento:**
   - Use `!nvidia-smi` para ver uso de GPU
   - Use `!htop` para ver uso de CPU/RAM

### ⚠️ Problemas Comuns

**Problema**: Map freezando em 9%
- **Solução**: Já aplicada! `num_proc=1` e cache habilitado

**Problema**: Out of Memory (OOM)
- **Solução**: Reduza batch_size ou reduce_ratio

**Problema**: GPUs não sendo usadas
- **Solução**: Verifique `devices=2` e `accelerator='gpu'` no Trainer

**Problema**: Treinamento muito lento
- **Solução**: Verifique se `precision='16-mixed'` está habilitado

## 📊 Resumo das Otimizações Aplicadas

### ✅ Melhorias no Código Base

| Arquivo | Mudança | Antes | Depois | Impacto |
|---------|---------|-------|--------|---------|
| `data.py` | num_proc reduzido | 4-8 | 1 | ⚡ Evita overhead e travamento |
| `data.py` | writer_batch_size | 500/32 | 100 | 💾 Reduz uso de memória |
| `data.py` | Cache habilitado | ❌ | ✅ | ⚡ Reutiliza processamento |
| `SynthGenerator.py` | num_proc reduzido | 8 | 1 | ⚡ Evita overhead |
| `configs` | num_workers | 20 | 4 | ⚡ Balanceado para 2 GPUs |
| `configs` | batch_size | 1 | 2 | ⚡ Melhor aproveitamento GPU |
| `configs` | reduce_ratio | 1.0 | 0.5 | 💾 Imagens 50% menores |

### ✅ Otimizações no Trainer

- **Multi-GPU**: DDP Strategy para 2 GPUs
- **Mixed Precision**: 16-bit (economiza ~50% de memória)
- **Gradient Accumulation**: 4 steps (batch efetivo = 16)
- **Gradient Clipping**: 1.0 (estabiliza treinamento)
- **Early Stopping**: Patience 10 épocas
- **Checkpointing**: Top-2 + último modelo

### 🎯 Resultado Esperado

Com estas otimizações:
- ✅ Map não deve mais travar
- ✅ Uso de memória otimizado (30GB RAM + 2x GPU)
- ✅ Treinamento ~2-3x mais rápido
- ✅ Batch efetivo maior = melhor convergência

## 📤 Git: Fazer Push das Otimizações

Use esta célula se você estiver trabalhando localmente e quiser fazer push das alterações para o GitHub.

In [ ]:
# Configure suas credenciais Git (apenas na primeira vez)
# !git config --global user.email "seu_email@example.com"
# !git config --global user.name "Seu Nome"

# Adicionar todas as alterações
!git add data.py SynthGenerator.py train_kaggle.py config/ KAGGLE_OPTIMIZATIONS.md kaggle_training_notebook.ipynb

# Fazer commit
!git commit -m "✨ Otimizações para Kaggle: fix Map freeze, multi-GPU, memory optimization"

# Push para o repositório
!git push origin main

print("✅ Alterações enviadas para o GitHub!")